In [6]:
import os
import json
import cv2
import pickle as pkl
import numpy as np
from numpy import random as rng
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage.io as ski
import PIL
from PIL import Image
import io

import tensorflow as tf
import tensorflow.keras as ks
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.data.experimental import AUTOTUNE

%matplotlib inline


In [2]:
data_path = '/mnt/d/cardimagescans'
tf.config.list_physical_devices()

2022-05-15 11:09:05.510945: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-15 11:09:05.816989: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-15 11:09:05.820635: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce GTX 1660 SUPER computeCapability: 7.5
coreClock: 1.83GHz coreCount: 22 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2022-05-15 11:09:05.820722: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-15 11:09:05.858584: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05-15 11:09:05.880151: I tensorflow/stream_ex

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# builds a dictionary containing all set codes and sizes of
# png files of cards in the database
# WARNING: Long runtime. Skip if set_dict.pickle is available
sets = os.listdir(data_path)
set_dict = {}
with open('set_dict.pickle', 'rb') as f:
    set_dict = pkl.load(f)
for s in sets:
    cardsinset = os.listdir(os.path.join(data_path, s))
    if s in set_dict.keys() and len(cardsinset) == len(set_dict.get(s)):
        pass
    else:
        set_dict[s] = []
        print('Listing set {}'.format(s))
        for card in cardsinset:
            image = cv2.imread(os.path.join(data_path, s, card))
            if type(image) == type(None):
                print('Error loading {} {}'.format(s, card))
            h, w = image.shape[:2]
            set_dict[s].append([h,w])

with open('set_dict.pickle', mode='wb') as f:
    pkl.dump(set_dict, f)

In [ ]:
# load set_dict.pickle to avoid long runtime
with open('set_dict.pickle', 'rb') as f:
    set_dict = pkl.load(f)

In [ ]:
# returns true if all card images have the same hxw size
def uniform_cardsize(setname, set_dict):
    s = np.array(set_dict.get(setname, False))
    if False in s:
        return False
    return np.size([list(set(s[:,0])), list(set(s[:,1]))]) == 2


In [3]:
# random horizontal cutoff
def random_h_shift(image, ratio=0.5):
    height, width = image.shape[:2]
    cutoff = rng.uniform(-ratio, ratio)
    if cutoff >0:
        shifted = image[:, :int(width-width*cutoff), :]
    if cutoff <0:
        shifted = image[:, int(-1*width*cutoff):, :]
    resized = cv2.resize(shifted, (width, height), cv2.INTER_CUBIC)
    return resized

# random vertical cutoff
def random_v_shift(image):
    img_height = image.shape[0]
    cutoff = rng.choice(range(0-int(img_height/2), int(img_height/2)))
    if cutoff >= 0:
        shifted = image[cutoff:]
    else:
        shifted = image[:cutoff]
    resized = cv2.resize(shifted, image.shape[1::-1], cv2.INTER_CUBIC)
    return resized

# randomly increase or decrease brightness
def random_brightness(image, low=.05, high=3):
    value = rng.uniform(low,high)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value # scale values
    hsv[:,:,1][hsv[:,:,1]>255] = 255 # clip high values
    hsv[:,:,2] = hsv[:,:,2]*value
    hsv[:,:,2][hsv[:,:,2]>255] = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

# add random blur
def random_blur(image, kernel_min=5, kernel_max=30):
    kernel_size = tuple(np.random.choice(range(kernel_min, kernel_max), size=2))
    img = cv2.blur(image, kernel_size, cv2.BORDER_DEFAULT)
    return img

# add random contrast changes
def random_contrast(image, low=0.5, high=3):
    #print(image[42])
    value = rng.uniform(low, high)
    #print(value)
    img = np.multiply(np.array(image).astype(np.int32), value)
    #print(np.equal(img, image))
    img[img>255] = 255
    return img.astype('uint8')


# TODO: implement image preprocessing
# def augment(image):
#     result = np.zeros((101,1040,745,3))
#     index = 0
#     for i in range(1,11):
#         result[index] = random_h_shift(image) # h_shift
#         index +=1
#         result[index] = random_v_shift(image) # v_shift
#         index +=1
#         result[index] = random_blur(image, kernel_min=2*i, kernel_max=5+2*i) # blur
#         index +=1
#         result[index] = random_h_shift(random_v_shift(image)) # h&v shift
#         index +=1
#     for i in range(5):
#         a = 1 + 3*i
#         b = 5 + 5*i
#         result[index] = random_brightness(image, 0.1, 0.99) # lower brightness
#         index +=1
#         result[index] = random_brightness(image, 1.1, 3) # higher brightness
#         index +=1
#         result[index] = random_blur(
#             random_brightness(image, 0.1, 0.99), a, b) # lower brightness + blur
#         index +=1
#         result[index] = random_blur(
#             random_brightness(image, 1.1, 3), a, b) # higher brightness + blur
#         index +=1
#         result[index] = random_contrast(image, 0.2, 0.99) # lower contrast
#         index +=1
#         result[index] = random_contrast(image, 1.01, 3) # higher contrast
#         index +=1
#         result[index] = random_blur(
#             random_contrast(image, 0.2, 0.99), a, b) # lower contrast + blur
#         index +=1
#         result[index] = random_blur(
#             random_contrast(image, 1.01, 3), a, b) # higher contrast + blur
#         index +=1
#         result[index] = random_v_shift(random_h_shift(
#             random_blur(random_contrast(image, 0.2, 0.99), a, b))) # lower contrast + blur + hv shift
#         index +=1
#         result[index] = random_v_shift(random_h_shift(
#             random_blur(random_contrast(image, 1.01, 3), a, b))) # higher contrast + blur + hv shift
#         index +=1
#         result[index] = random_h_shift(random_v_shift(
#             random_blur(random_brightness(image, 0.1, 0.99), a, b) # lower brightness + blur + hv shift
#         ))
#         index +=1
#         result[index] = random_h_shift(random_v_shift(
#             random_blur(random_brightness(image, 1.1, 3), a, b) # higher brightness + blur + hv shift
#         ))
#         index +=1
#     result[index] = image # append original image as the 101st in the dataset
#     return np.array(result)

# apply a random augmentation to the image
# intended to be a simple, reusable function that can be applied 
# multiple times to increase difficulty
def random_augment(image, choice=False):
    if not choice:
        choice = rng.choice(range(8))
    if choice == 0:
        return random_h_shift(image)
    if choice == 1:
        return random_v_shift(image)
    if choice == 2:
        return random_brightness(image, low= 0.05, high=0.99)
    if choice == 3:
        return random_brightness(image, low=1.01, high=3)
    if choice == 4:
        return random_blur(image, kernel_min=5, kernel_max=15)
    if choice == 5:
        return random_blur(image, kernel_min=15, kernel_max=30)
    if choice == 6:
        return random_contrast(image, low=0.2, high=0.8)
    if choice == 7:
        return random_contrast(image, low=1.2, high=3)

def augment(image, runs=3):
    img = image
    for i in range(runs):
        img = random_augment(img)
    return img



# TODO: implement dataset generation using preprocessing
def make_dataset(setcodes, data_path):
    data = []
    labels = []
    for code in setcodes:
        set_dir = os.listdir(os.path.join(data_path, code))
        for card in set_dir:
            image = cv2.imread(os.path.join(data_path, code, card))
            labels.append(image)
            data.append(augment(image))
    assert len(data) == len(labels), 'Length of data and labels does not match: {} vs {}'.format(len(data), len(labels))
    return (data, labels)


# Quickly display a card
def display_card(image, size=(12,16)):
    fig, ax = plt.subplots(figsize=size)
    ax.imshow(random_contrast(image, 0.5, 3))
    plt.show()

In [7]:
def build_tfrecords(datapath, setcode):
    with tf.io.TFRecordWriter('{}.tfrecords'.format(setcode)) as writer:
        cards = os.listdir(os.path.join(datapath, setcode))
        labels = [i for i in range(len(cards))]

        for path, label in zip(cards, labels):
            image = Image.open(os.path.join(datapath, setcode, path))
            bytes_buffer = io.BytesIO()
            image.convert('RGB').save(bytes_buffer, 'JPEG')
            image_bytes = bytes_buffer.getvalue()

            bytes_feature = tf.train.Feature(bytes_list = tf.train.BytesList(value=[image_bytes]))
            class_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))

            example = tf.train.Example(
                features = tf.train.Features(feature={
                    'image': bytes_feature,
                    'class': class_feature
                })
            )
            writer.write(example.SerializeToString())
            image.close()

image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, [256, 256, 3])
    return image, parsed_example["class"]

def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

In [8]:
build_tfrecords(data_path, 'khm')
ds_train = load_and_extract_images('khm.tfrecords')

In [9]:
print(ds_train)

<ParallelMapDataset shapes: ((256, 256, 3), ()), types: (tf.uint8, tf.int64)>


In [ ]:
display_card(image)

In [ ]:
# quickly test dataset generation

height = 1040
width = 745

model = ks.Sequential([
    layers.Input((height, width, 3)),
    layers.Conv2D(16, 3, padding='same'),
    layers.Conv2D(32, 3, padding='same'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(10)
])

In [ ]:
def create_dataset_from_directory(datadir):
    files = os.listdir(datadir)
    training_data = []
    for i,f in enumerate(files):
        filepath = os.path.join(datadir, f)
        img_array = ski.imread(filepath)
        training_data.append([img_array, i])
    return training_data

In [ ]:
ds_train = create_dataset_from_directory(os.path.join(data_path, 'khm'))

In [ ]:
print(len(ds_train))

In [ ]:
ds_train = ks.preprocessing.image_dataset_from_directory(
    '/mnt/d/cardimagescans/khm',
    labels = 'inferred',
    label_mode = 'int',
    color_mode = 'rgb',
    batch_size = 8,
    image_size = (height, width),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='training'
)

ds_train.map(augment)

In [ ]:
dataset = make_dataset(['khm'], data_path)


In [ ]:
test_set_path = os.path.join(data_path, 'khm')
test_set = os.listdir(test_set_path)
test_img = cv2.imread(os.path.join(test_set_path, test_set[69]))

In [ ]:
shifted = random_contrast(test_img)
print(shifted.shape)
display_card(shifted)

In [ ]:
single_card_set = augment(test_img)

In [ ]:
for i, card in enumerate(single_card_set):
    if i%9==0:
        display_card(card, (3,4))

In [ ]:


fig, ax = plt.subplots(figsize=(12,16))
ax.imshow(random_contrast(test_img, 0.5, 3))
plt.show()

In [ ]:
print(np.shape(set_dict.get('bfz')))
for k in set_dict.keys():
    s =  np.array(set_dict.get(k))
    h = set(s[:,0])
    w = set(s[:,1])
    print(k, h, w)